In [1]:
import numpy as np
import torch
from utils.motion import *
from utils.skeleton import Skeleton

In [26]:
data = np.load("/source/junhyuk/motion-style/results/4/reconstruct/030075_reconstructed.npy")
# data = np.load("/source/junhyuk/motion-style/dataset/100style/new_joint_vecs/030001.npy")
data = torch.from_numpy(data).float()

In [27]:
joint_pos = recover_from_ric(data, 22)
# np.save("./test_gt.npy", joint_pos)

In [17]:
skel = np.load("./smpl22_offsets.npz", allow_pickle=True)
# offset = torch.tensor(skel["offsets"], dtype=torch.float32)
lengths = skel["lengths"]
parents = skel["parents"]

In [4]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# === Default SMPL-22 parent list (index-aligned) ===
# -1 means "no parent" (root)
SMPL22_PARENTS = [
    -1,  # 0 pelvis
     0,  # 1 left_hip
     0,  # 2 right_hip
     0,  # 3 spine1
     1,  # 4 left_knee
     2,  # 5 right_knee
     3,  # 6 spine2
     4,  # 7 left_ankle
     5,  # 8 right_ankle
     6,  # 9 spine3
     7,  # 10 left_foot
     8,  # 11 right_foot
     9,  # 12 neck
    12,  # 13 left_collar
    12,  # 14 right_collar
    12,  # 15 head
    13,  # 16 left_shoulder
    14,  # 17 right_shoulder
    16,  # 18 left_elbow
    17,  # 19 right_elbow
    18,  # 20 left_wrist
    19,  # 21 right_wrist
]

def fk_from_offsets(offsets, parents, assume_local=True):
    """Return world/rest positions from offsets.
       If assume_local=True, treats offsets[j] as the vector from parent(j) to j."""
    offsets = np.asarray(offsets, float)
    J = offsets.shape[0]
    if not assume_local:
        return offsets.copy()
    pos = np.zeros_like(offsets)
    # Root: most formats store [0,0,0]; keep whatever is given
    pos[0] = offsets[0]
    for j in range(1, J):
        p = parents[j]
        pos[j] = pos[p] + offsets[j]
    return pos

def visualize_offsets(offsets, parents=None, annotate=True, y_up=False, assume_local=True, center_root=True, invert_z_axis=True):
    offsets = np.asarray(offsets, float)
    assert offsets.ndim == 2 and offsets.shape[1] == 3, "offsets must be [J,3]"

    # 1) Convert local offsets -> world positions
    P = fk_from_offsets(offsets, parents, assume_local=assume_local) if parents is not None else offsets.copy()

    # Optional: center root to origin for a cleaner view
    if center_root:
        P = P - P[0:1]

    # 2) Y-up swap only for plotting
    plot_xyz = P[:, [0, 2, 1]] if y_up else P

    fig = plt.figure()
    ax = fig.add_subplot(111, projection="3d")
    ax.scatter(plot_xyz[:, 0], plot_xyz[:, 1], plot_xyz[:, 2])

    if parents is not None:
        for j, p in enumerate(parents):
            if p >= 0:
                ax.plot([plot_xyz[p,0], plot_xyz[j,0]],
                        [plot_xyz[p,1], plot_xyz[j,1]],
                        [plot_xyz[p,2], plot_xyz[j,2]])

    if annotate:
        for i, (x, y, z) in enumerate(plot_xyz):
            ax.text(x, y, z, str(i))

    mins, maxs = plot_xyz.min(0), plot_xyz.max(0)
    center, radius = (mins+maxs)/2.0, (maxs-mins).max()/2.0
    ax.set_xlim(center[0]-radius, center[0]+radius)
    ax.set_ylim(center[1]-radius, center[1]+radius)
    ax.set_zlim(center[2]-radius, center[2]+radius)
    try: ax.set_box_aspect([1,1,1])
    except: pass

    ax.set_xlabel("X")
    ax.set_ylabel("Z" if y_up else "Y")
    ax.set_zlabel("Y (up)" if y_up else "Z (up)")
    ax.set_title("Rest Pose — " + ("Y-up" if y_up else "Z-up") + (" (local→world)" if assume_local else " (absolute)"))
    plt.show()

# Example:
visualize_offsets(offsets2, parents=SMPL22_PARENTS, annotate=True, y_up=True, assume_local=True)

NameError: name 'offsets2' is not defined

In [5]:
visualize_offsets(offsets, parents=SMPL22_PARENTS, annotate=True, y_up=True, assume_local=True)

NameError: name 'offsets' is not defined

In [48]:
# idk = np.array(
#     [
#         [0, 0, 0],
#         [1, 0, 0],
#         [-1 ,0, 0],
#         [0, 1, 0],
#         [0, -1, 0],
#         [0, -1, 0],
#         [0, 1, 0],
#         [0, -1, 0],
#         [0, -1, 0],
#         [0, 1, 0],
#         [0, 0, 1],
#         [0, 0, 1],
#         [0, 1, 0],
#         [1, 0, 0],
#         [-1, 0, 0],
#         [0, 0, 1],
#         [0, -1, 0],
#         [0, -1, 0],
#         [0, -1, 0],
#         [0, -1, 0],
#         [0, -1, 0],
#         [0, -1, 0]
#     ]
# )

In [49]:
# parents = np.array([
#     -1,  # 0 pelvis
#      0,  # 1 left_hip
#      0,  # 2 right_hip
#      0,  # 3 spine1
#      1,  # 4 left_knee
#      2,  # 5 right_knee
#      3,  # 6 spine2
#      4,  # 7 left_ankle
#      5,  # 8 right_ankle
#      6,  # 9 spine3
#      7,  # 10 left_foot
#      8,  # 11 right_foot
#      9,  # 12 neck
#      9,  # 13 left_collar
#      9,  # 14 right_collar
#     12,  # 15 head
#     13,  # 16 left_shoulder
#     14,  # 17 right_shoulder
#     16,  # 18 left_elbow
#     17,  # 19 right_elbow
#     18,  # 20 left_wrist
#     19,  # 21 right_wrist
# ], dtype=np.int32)

In [18]:
skeleton = Skeleton(parents, lengths)

In [62]:
# skeleton = Skeleton(offset, kinematic_tree, device='cpu')
# skeleton.set_offset(offset)

In [20]:
joint_pos2 = recover_from_rot(data, 22, skeleton).numpy()
np.save("./test_rot.npy", joint_pos2)

In [16]:
joint_pos[0]

tensor([[ 0.0000,  0.9403,  0.0000],
        [ 0.0613,  0.8623, -0.0246],
        [-0.0617,  0.8515, -0.0143],
        [-0.0071,  1.0701, -0.0062],
        [ 0.1176,  0.4926,  0.0888],
        [-0.1209,  0.4694, -0.0090],
        [-0.0139,  1.2228,  0.0398],
        [ 0.1220,  0.0715,  0.1076],
        [-0.0539,  0.1622, -0.2014],
        [-0.0053,  1.2598,  0.0779],
        [ 0.1537,  0.0177,  0.2304],
        [-0.0830,  0.0543, -0.1627],
        [ 0.0091,  1.4875,  0.1160],
        [ 0.0750,  1.3792,  0.0767],
        [-0.0703,  1.3911,  0.0951],
        [ 0.0291,  1.5502,  0.1916],
        [ 0.2023,  1.4053,  0.0725],
        [-0.1873,  1.4183,  0.1146],
        [ 0.4207,  1.2381,  0.0463],
        [-0.4080,  1.2644,  0.1222],
        [ 0.6514,  1.1134,  0.0692],
        [-0.6240,  1.0861,  0.1270]])

In [28]:
kinematic_chain = [[0, 2, 5, 8, 11], [0, 1, 4, 7, 10], [0, 3, 6, 9, 12, 15], [9, 14, 17, 19, 21], [9, 13, 16, 18, 20]]

In [30]:
import os
from salad.utils.plot_script import plot_3d_motion
save_path = os.path.join("./delete2.mp4")
plot_3d_motion(save_path, kinematic_chain, joint_pos, title='test', fps=20);

AttributeError: 'Tensor' object has no attribute 'copy'

In [105]:
from glob import glob

device = 'cuda:0'
opt = get_opt("checkpoints/t2m/t2m_denoiser_vpred_vaegelu/opt.txt", device)

input_folder = "/source/junhyuk/motion-style/results/4/style"
output_folder = "video/4/style"
os.makedirs(output_folder, exist_ok=True)

npy_files = sorted(glob(os.path.join(input_folder, "*.npy")))

for npy_file in npy_files:
    filename = os.path.basename(npy_file)
    basename = os.path.splitext(filename)[0]  # e.g., "030075_reconstructed"

    data = np.load(npy_file)
    joint = recover_from_ric(torch.from_numpy(data).float(), opt.joints_num).numpy()

    save_path = os.path.join(output_folder, f"{basename}.mp4")
    plot_3d_motion(save_path, opt.kinematic_chain, joint, title=basename, fps=20)

    print("✅ Saved", save_path)

NameError: name 'get_opt' is not defined

In [35]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def show_frame_by_frame(joint_seq, skeleton):
    for t in range(joint_seq.shape[0]):
        joints = joint_seq[t]

        fig = plt.figure(figsize=(6, 6))
        ax = fig.add_subplot(111, projection='3d')
        ax.set_title(f"Frame {t}")
        ax.set_xlim(-1, 1)
        ax.set_ylim(-1, 1)
        ax.set_zlim(0, 2)
        ax.set_xlabel("X")
        ax.set_ylabel("Z")
        ax.set_zlabel("Y")
        ax.view_init(elev=110, azim=-90)

        # Draw joints and bones
        ax.scatter(joints[:, 0], joints[:, 2], joints[:, 1], c='blue', s=20)
        for i, j in skeleton:
            ax.plot(
                [joints[i, 0], joints[j, 0]],
                [joints[i, 2], joints[j, 2]],
                [joints[i, 1], joints[j, 1]],
                c='blue', linewidth=2
            )

        plt.show()
        input("Press Enter for next frame...")
        plt.close(fig)